## Step 1 : Import Data

In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load your Excel data
excel_data = pd.ExcelFile("Group-Project_Covid-Data.xlsx")

# Reading each sheet into a pandas DataFrame
sheets = {}
for sheet_name in excel_data.sheet_names:
    sheets[sheet_name] = pd.read_excel(excel_data, sheet_name=sheet_name)

# Prepare the data from the sheets for use in the model

# Vaccine Supply
supply = sheets['Vaccine Supply'].set_index('Supplier/t').T.to_dict('list')

# Demand Data
demand = sheets['Demand Data'].set_index('Neighborhood/t').T.to_dict('list')

# Transport Costs
transport_costs_df = sheets['Transport Costs']
transport_costs = {center: {} for center in transport_costs_df.columns[1:]}  # Exclude 'Supplier' column
for _, row in transport_costs_df.iterrows():
    supplier = row['Supplier']
    for center in transport_costs:
        transport_costs[center][supplier] = row[center]

# Vaccine Purchase Cost
vaccine_purchase_df = sheets['Vaccine Purchase Cost']
vaccine_cost = dict(zip(vaccine_purchase_df['Supplier'], vaccine_purchase_df['Cost']))


# Case Data
cases = sheets['Case Data'].set_index('Neighborhood/t').T.to_dict('list')

# Death Data
deaths = sheets['Death Data'].set_index('Neighborhood/t').T.to_dict('list')

# Center to Neighborhood Mapping
center_neighbourhood_df = sheets['Center to Neighbourhood']
neighbourhood_vacc_centers = {}
for index, row in center_neighbourhood_df.iterrows():
    neighbourhood = row['Neighbourhood']
    vacc_centers = [column for column in center_neighbourhood_df.columns[1:] if row[column] == 1]
    neighbourhood_vacc_centers[neighbourhood] = vacc_centers


# Parameters
parameters = sheets['Parameters'].set_index('Parameter Name')['Value'].to_dict()
h = parameters['h']
alpha = parameters['alpha']
beta = parameters['beta']

## Step 2 : Check that all data was properly uploaded

In [ ]:
# Print out the key data structures to verify the data import

# Vaccine Supply
print("Vaccine Supply:")
print(supply)

# Demand Data
print("\nDemand Data:")
print(demand)

# Transport Costs
print("\nTransport Costs:")
for center, costs in transport_costs.items():
    print(f"{center}: {costs}")

# Vaccine Purchase Cost
print("\nVaccine Purchase Cost:")
print(vaccine_cost)

# Case Data
print("\nCase Data:")
print(cases)

# Death Data
print("\nDeath Data:")
print(deaths)

# Center to Neighborhood Mapping
for neighbourhood, centers in neighbourhood_vacc_centers.items():
    print(f"\nNeighbourhood: {neighbourhood}, Vaccination Centers: {centers}")

# Parameters
print("\nParameters:")
print(parameters)

## Step 3 : Create lists of values

In [ ]:
# Set of suppliers, vaccination centers, and time periods
suppliers = list(supply.keys())  # e.g., ['Moderna', 'Pfizer-BioNTech', ...]
vaccination_centers = list(transport_costs.keys())  # e.g., ['Kirkland', 'Lachine', ...]
neighbourhoods = list(demand.keys())
time_periods = list(range(0, 5)) 

print(suppliers)
print(vaccination_centers)
print(neighbourhoods)
print(time_periods)

## Step 4: Create Model

In [ ]:
# Create a new model
model = gp.Model("VaccineDistribution")

## Step 5: Add Decision Variables

In [ ]:
# Decision Variables
X = model.addVars(suppliers, vaccination_centers, time_periods, vtype=GRB.INTEGER, name="X")

## Step 6: Create Objective Function Components

In [ ]:
# Transportation Costs Component
transportation_and_vaccine_costs = gp.quicksum(
    (transport_costs[i][j] + vaccine_cost[j]) * X[j, i, t] 
    for j in suppliers 
    for i in vaccination_centers 
    for t in time_periods
)

In [ ]:
urgency_penalty = gp.quicksum(
    (alpha * cases[n][t] + beta * deaths[n][t]
    for n in neighbourhood_vacc_centers
    for t in time_periods
))

# General Unmet demand
penalty_unmet_demand = gp.quicksum(
    (h+urgency_penalty) * (demand[n][t] - gp.quicksum(X[j, i, t] for i in neighbourhood_vacc_centers[n] for j in suppliers))
    for n in neighbourhood_vacc_centers
    for t in time_periods
)

In [ ]:
model.setObjective(transportation_and_vaccine_costs + penalty_unmet_demand, GRB.MINIMIZE)

# Update the Model
model.update()

## Step 7: Add Constraints

In [ ]:
# Supply Constraints
for j in suppliers:
    for t in time_periods:
        model.addConstr(
            gp.quicksum(X[j, i, t] for i in vaccination_centers) <= supply[j][t],
            name=f"SupplyConstraint_{j}_{t}"
        )

# Demand Constraints
for n in neighbourhood_vacc_centers:
    for t in time_periods:
        model.addConstr(
            gp.quicksum(X[j, i, t] for i in neighbourhood_vacc_centers[n] for j in suppliers) <= demand[n][t],
            name=f"DemandConstraint_{n}_{t}"
        )

# Constraints to Ensure 10% Proportion of Demand Met for All Neighborhoods
for n in neighbourhoods:
    for t in time_periods:
        total_vaccines_to_n = gp.quicksum(X[j, i, t] for i in neighbourhood_vacc_centers[n] for j in suppliers)
        model.addConstr(total_vaccines_to_n >= 0.10 * demand[n][t], name=f"ProportionDemandMet_{n}_{t}")
    
# Non-negativity Constraints
for i in vaccination_centers:
    for j in suppliers:
        for t in time_periods:
            model.addConstr(
                X[j, i, t] >= 0,
                name=f"NonNegativity_{j}_{i}_{t}"
            )

# Use all of the supply when supply < demand constraint
for t in time_periods:
    # Binary variable: 1 if total supply is less than total demand, 0 otherwise
    supply_less_than_demand = model.addVar(vtype=GRB.BINARY, name=f"SupplyLessThanDemand_{t}")

    total_supply_t = gp.quicksum(supply[j][t] for j in suppliers)
    total_demand_t = gp.quicksum(demand[n][t] for n in neighbourhoods)

    # Update the model to include the new variable
    model.update()

    # Adding indicator constraint
    model.addGenConstrIndicator(supply_less_than_demand, 1, total_supply_t <= total_demand_t)
    model.addConstr(
        gp.quicksum(X[j, i, t] for j in suppliers for i in vaccination_centers) <= total_supply_t,
        name=f"UseAllSupplyWhenLess_{t}"
    )
    model.addConstr(
        (supply_less_than_demand == 1) >> (gp.quicksum(X[j, i, t] for j in suppliers for i in vaccination_centers) == total_supply_t)
    )

# Equal Distribution Constraints
for n in neighbourhoods:
    for t in time_periods:
        total_vaccines_for_neighborhood = gp.quicksum(X[j, i, t] for i in neighbourhood_vacc_centers[n] for j in suppliers)
        num_centers_in_neighborhood = len(neighbourhood_vacc_centers[n])
        
        for i in neighbourhood_vacc_centers[n]:
            model.addConstr(
                X.sum('*', i, t) == total_vaccines_for_neighborhood / num_centers_in_neighborhood,
                name=f"EqualDistribution_{n}_{i}_{t}"
            )

# Update the Model with these constraints
model.update()

## Step 8: Solve Model

In [ ]:
# Solve the model
model.optimize()

In [ ]:
# Check if the model has been solved to optimality
if model.status == GRB.OPTIMAL:
    # Retrieve the results
    print('Solution status is optimal, and the minimum cost is: ${:.4f}'.format(model.objVal))
    transportation_and_vaccine_costs_value = transportation_and_vaccine_costs.getValue()
    print('Total Procurement and Transportation costs: ${:.2f}'.format(transportation_and_vaccine_costs_value))
    indirect_costs=model.objVal - transportation_and_vaccine_costs_value
    print('Total Indirect costs: ${:.2f}'.format(indirect_costs))
    
    vaccine_distribution = model.getAttr('X', X)

    # Print the results in a table-like format
    print(f"{'Supplier':<25} {'Vaccination Center':<25} {'Time Period':<15} {'Vaccines Distributed':<20}")
    print("-" * 85)
    for (j, i, t), value in vaccine_distribution.items():
        if value > 0:  # To only display non-zero distributions
            print(f"{j:<25} {i:<25} {t:<15} {value:<20}")

elif model.status == GRB.INFEASIBLE:
    print("Model is infeasible. No solution found.")
else:
    print("Optimization was stopped with status", model.status)

## Step 9: Sensitivity Analysis

In [ ]:
# Sensitivity Analysis

if model.status == GRB.OPTIMAL:
    print('Solution status is optimal, and the minimum cost is: ${:.4f}'.format(model.objVal))
    header = "| {:<40} | {:<6} | {:<10} | {:<10} |".format(
        "Constraint Name", "Sense", "Slack", "RHS"
    )
    print(header)
    print("-" * len(header))  # Print a separator line
    for constr in model.getConstrs():
        name = constr.ConstrName
        sense = constr.Sense
        slack = constr.Slack
        rhs = constr.RHS

        # Only print if the constraint has non-zero slack or RHS
        if slack != 0 or rhs != 0:
            row = "| {:<40} | {:<6} | {:<10.2f} | {:<10.2f} |".format(
                name, sense, slack, rhs
            )
            print(row)
else:
    print('Optimization was stopped with status %d' % model.status)

## Plot for sensitivity analysis

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Data for the comprehensive graph
data_complete = {
    "Week": [0, 1, 2, 3, 4],
    "Moderna Surplus": [173, 0, 0, 573, 1307],
    "Novavax Surplus": [0, 0, 0, 0, 605],
    "Pfizer-BioNTech Surplus": [0, 0, 0, 0, 0],
    "Johnson & Johnson (Janssen) Surplus": [0, 0, 0, 0, 0],
    "AstraZeneca Surplus": [0, 0, 0, 0, 0],
    "MedicagoSanofi and GlaxoSmithKline Surplus": [0, 0, 0, 0, 0],
    "l'Ouest-de-l'Île-de-Montréal Slack": [0.65, 1.60, 1.91, 0.15, 1.70],
    "Centre-Sud-de-l'Île-de-Montréal Slack": [1.37, 298.57, 320.81, 1.02, 0.65],
    "Centre-Ouest-de-l'Île-de-Montréal Slack": [2.51, 1.47, 1.78, 0.94, 1.57],
    "Nord-de-l'Île-de-Montréal Slack": [1.69, 1.45, 35.52, 1.37, 1.48],
    "l'Est-de-l'Île-de-Montréal Slack": [0.48, 175.97, 227.42, 2.19, 2.12],
    "Montérégie Slack": [0.78, 1.78, 1.69, 0.41, 0.73],
    "Laval Slack": [0.14, 598.74, 602.69, 0.98, 0.08],
    "Laurentides Slack": [0.31, 863.33, 863.16, 0.78, 0.48],
    "Lanaudière Slack": [0.23, 0.81, 0.03, 0.27, 0.85]
}

df_complete = pd.DataFrame(data_complete)

# Creating the plot
plt.figure(figsize=(15, 8))

# Red shades for vaccines
red_shades = plt.cm.get_cmap('Reds', 16)
for i, vaccine in enumerate(df_complete.columns[1:7]):
    plt.plot(df_complete["Week"], df_complete[vaccine], marker='o', color=red_shades(i + 4), label=vaccine)

# Adjusted colors for regions
region_colors = ['maroon', 'darkred', 'red', 'firebrick', 'crimson', 'salmon', 'lightcoral', 'indianred', 'brown', 'tomato']
for i, region in enumerate(df_complete.columns[7:]):
    plt.plot(df_complete["Week"], df_complete[region], marker='x', linestyle='--', color=region_colors[i], label=region)

plt.title("Comprehensive View of COVID-19 Vaccine Supply Surplus and Demand Slack in Montreal")
plt.xlabel("Week")
plt.ylabel("Units")
plt.legend(loc='upper left', ncol=2)
plt.grid(True)
plt.show()

## Heatmap for sensitivity analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Data preparation
data_complete = {
    "Week": [0, 1, 2, 3, 4],
    "Moderna Surplus": [173, 0, 0, 573, 1307],
    "Novavax Surplus": [0, 0, 0, 0, 605],
    "Pfizer-BioNTech Surplus": [0, 0, 0, 0, 0],  # Assumed zero
    "Johnson & Johnson (Janssen) Surplus": [0, 0, 0, 0, 0],  # Assumed zero
    "AstraZeneca Surplus": [0, 0, 0, 0, 0],  # Assumed zero
    "MedicagoSanofi and GlaxoSmithKline Surplus": [0, 0, 0, 0, 0],  # Assumed zero
    "l'Ouest-de-l'Île-de-Montréal Slack": [0.65, 1.60, 1.91, 0.15, 1.70],
    "Centre-Sud-de-l'Île-de-Montréal Slack": [1.37, 298.57, 320.81, 1.02, 0.65],
    "Centre-Ouest-de-l'Île-de-Montréal Slack": [2.51, 1.47, 1.78, 0.94, 1.57],
    "Nord-de-l'Île-de-Montréal Slack": [1.69, 1.45, 35.52, 1.37, 1.48],
    "l'Est-de-l'Île-de-Montréal Slack": [0.48, 175.97, 227.42, 2.19, 2.12],
    "Montérégie Slack": [0.78, 1.78, 1.69, 0.41, 0.73],
    "Laval Slack": [0.14, 598.74, 602.69, 0.98, 0.08],
    "Laurentides Slack": [0.31, 863.33, 863.16, 0.78, 0.48],
    "Lanaudière Slack": [0.23, 0.81, 0.03, 0.27, 0.85]
}
df_complete = pd.DataFrame(data_complete)

# Creating heatmap data
heatmap_data = df_complete.drop("Week", axis=1).T
heatmap_data.columns = [f"Week {week}" for week in heatmap_data.columns]

# Plotting the heatmap
plt.figure(figsize=(10, 8))
plt.title("Heatmap of Vaccine Surplus and Demand Slack in Montreal")
sns.heatmap(heatmap_data, cmap="YlOrRd", annot=True, fmt=".1f")
plt.xlabel("Week")
plt.ylabel("Vaccine/Region")
plt.show()